In [1]:
import re
from hashlib import sha256
import requests
import datetime
from pathlib import Path

from fetchers.FetchConfig import FetchConfig
from fetchers import DiscoverDownloadVerifyUnpackFetcher, GitRepoFetcher
from fetchers.discoverers.GitHub import GitHubReleasesDiscoverer
from fetchers.discoverers.DNF import DNFDiscoverer
from fetchers.unpackers.compression.gzip import GZIP
from fetchers.unpackers.archives import Archive
from fetchers.unpackers.archives.tar import Tar
from fetchers.verifiers.OpenPGP import OpenPGPVerifier
from fetchers.verifiers.hashlist import HashlistVerifier

thisDir = Path(".").absolute()
cfg = FetchConfig(downloadsTmp=thisDir / "downloads")

# Downloading source code from a Git repo

In [2]:
moduleMdFetcher = GitRepoFetcher("https://github.com/fedora-modularity/libmodulemd", refspec="main")
fetchedInfo = moduleMdFetcher(thisDir / "modulemd", cfg)  # for this one config is not necessary

Cloning a git repo: 0it [00:00, ?it/s]

Detected service <fetchers.webServices.services.GitHub.GitHubService object at 0x7fde88e94ca0>


52 / 60 86.66666666666667% limit will be reset: 2021-03-11 15:35:47 in -1 day, 21:02:02.950156
Cloning https://github.com/fedora-modularity/libmodulemd to /home/KOLANICH/fetchers.py/modulemd ...


Cloned


version from tag: 2.12.0


In [3]:
print(repr(fetchedInfo.version))
print(fetchedInfo.version)
print(fetchedInfo.version.major)
print(fetchedInfo.version.minor)
print(fetchedInfo.version.patch)
print(fetchedInfo.fetcherID)

AnyVer('2.12.0')
2.12.0
2
12
0
Aip6mTFwGQHpnJUOJCTQKQ==


In [4]:
datetime.datetime.fromtimestamp(fetchedInfo.time)

datetime.datetime(2021, 1, 15, 5, 9, 23)

# Downloading binary builds from GitHub releases
(Tar unpacker has a minor bug in progress reporter: it overestimates the size of files in the tarball a bit, that's why the progressbar is red)

In [5]:
# Getting Bradley King's key (you should ship the key, I just don't want to junk the repo, but the lib can also accept `bytes`) and saving it
bradKeyResp = requests.get("https://keyserver.ubuntu.com/pks/lookup?op=get&search=0xcba23971357c2e6590d9efd3ec8fef3a7bfb4eda")
bradKeyPath = thisDir / "brad.gpg"
bradKeyPath.write_bytes(bradKeyResp.content)

35911

In [6]:
versionRxText = "(?:\\d+\\.){1,2}\\d+(?:-rc\\d+)?"
platformMarker = "Linux-x86_64"
hashFuncName = "SHA-256"
downloadFileNameRx = re.compile("^" + "-".join(("cmake", versionRxText, platformMarker)) + "\\.tar\\.gz$")
hashesFileNameRxText = "-".join(("cmake", versionRxText, hashFuncName)) + "\\.txt"
hashesSigFileNameRxText = hashesFileNameRxText + "\\.(?:asc|sig|gpg)"

cmakeFetcher = DiscoverDownloadVerifyUnpackFetcher(
	GitHubReleasesDiscoverer(
		"Kitware/CMake", tagRx=re.compile("^v(" + versionRxText + ")$"),
		downloadFileNamesRxs={
			"binary.tgz": downloadFileNameRx,
			"hashes": re.compile("^" + hashesFileNameRxText + "$"),
			"hashes.asc": re.compile("^" + hashesSigFileNameRxText + "$"),
		},
		titleRx = None
	),
	HashlistVerifier(OpenPGPVerifier, sha256, hashesFileName="hashes",  signatureFileName="hashes.asc", keyFingerprint = "CBA23971357C2E6590D9EFD3EC8FEF3A7BFB4EDA", keyFile=bradKeyPath),
	Archive("binary.tgz", Tar, GZIP)
)
fetchedInfo = cmakeFetcher("./cmake", cfg)

51 / 60 85.0% limit will be reset: 2021-03-11 15:35:47 in -1 day, 21:02:00.475356


Selected release:  [3.19.6, release] <{'binary.tgz': DownloadTargetFile('binary.tgz', 'cmake-3.19.6-Linux-x86_64.tar.gz', datetime.datetime(2021, 2, 24, 16, 19, 50, tzinfo=tzutc()), datetime.datetime(2021, 2, 24, 16, 20, 5, tzinfo=tzutc()), 'https://github.com/Kitware/CMake/releases/download/v3.19.6/cmake-3.19.6-Linux-x86_64.tar.gz', 42650953), 'hashes': DownloadTargetFile('hashes', 'cmake-3.19.6-SHA-256.txt', datetime.datetime(2021, 2, 24, 16, 21, 32, tzinfo=tzutc()), datetime.datetime(2021, 2, 24, 16, 21, 33, tzinfo=tzutc()), 'https://github.com/Kitware/CMake/releases/download/v3.19.6/cmake-3.19.6-SHA-256.txt', 1343), 'hashes.asc': DownloadTargetFile('hashes.asc', 'cmake-3.19.6-SHA-256.txt.asc', datetime.datetime(2021, 2, 24, 16, 22, 3, tzinfo=tzutc()), datetime.datetime(2021, 2, 24, 16, 22, 3, tzinfo=tzutc()), 'https://github.com/Kitware/CMake/releases/download/v3.19.6/cmake-3.19.6-SHA-256.txt.asc', 833)}>


downloadedTargets {'binary.tgz': DownloadTarget('binary.tgz', 'cmake-3.19.6-Linux-x86_64.tar.gz', datetime.datetime(2021, 2, 24, 16, 20, 5, tzinfo=tzutc())), 'hashes': DownloadTarget('hashes', 'cmake-3.19.6-SHA-256.txt', datetime.datetime(2021, 2, 24, 16, 21, 33, tzinfo=tzutc())), 'hashes.asc': DownloadTarget('hashes.asc', 'cmake-3.19.6-SHA-256.txt.asc', datetime.datetime(2021, 2, 24, 16, 22, 3, tzinfo=tzutc()))}
hashes {'cmake-3.19.6-Linux-aarch64.sh': 'f383c2ef96e5de47c0a55957e9af0bdfcf99d3988c17103767c9ef1b3cd8c0a9', 'cmake-3.19.6-Linux-aarch64.tar.gz': '69ec045c6993907a4f4a77349d0a0668f1bd3ce8bc5f6fbab6dc7a7e2ffc4f80', 'cmake-3.19.6-Linux-x86_64.sh': 'd94155cef56ff88977306653a33e50123bb49885cd085edd471d70dfdfc4f859', 'cmake-3.19.6-Linux-x86_64.tar.gz': '139580473b84f5c6cf27b1d1ac84e9aa6968aa13e4b1900394c50075b366fb15', 'cmake-3.19.6-macos-universal.dmg': 'f840e573493ce41c5ff3bc389a7cba6fcc4ec758de612fe2195552258b6ead43', 'cmake-3.19.6-macos-universal.tar.gz': 'c0b73e5871e82153f0bce

unpacking archive:   0%|		  | 0/121725440 [00:00<?, ?it/s]

In [7]:
cmakeFetcher.id

'Y83m5Z6NlrxEbpJ3IIHf4Q=='

In [8]:
fetchedInfo.version

AnyVer('3.19.6')

In [9]:
fetchedInfo.time

datetime.datetime(2021, 2, 24, 16, 22, 3, tzinfo=tzutc())